In [1]:
import torch
from torch.autograd import Variable
from torch import nn
import numpy as np
from itertools import chain

In [ ]:
source_file = "data/es.txt"
target_file = "data/en.txt"
source_lines = list(open(source_file, "r"))
target_lines = list(open(target_file, "r"))

# very crude
s="abcdefghijklmnopqrstuvwxyz"

training_pairs = zip(source_lines, target_lines)

max_seq_len = 30
keep_pair_if = lambda pair: len(pair[0].split(" ")) < max_seq_len and len(pair[1].split(" ")) < max_seq_len

training_pairs = filter(keep_pair_if, training_pairs)
source_lines, target_lines = zip(*training_pairs)

class Corpus():
    def __init__(self, input_lines, n_train=5000):
        self.SOS = 0
        self.EOS = 1
        self.idx_word, self.word_idx = self.parse_words(input_lines)
        self.n_train = n_train
        
        self.parse_words(input_lines)
        self.corpus_size = len(self.idx_word)
        self.lines = [l.strip().lower() for l in input_lines]
        self.training = [self.sentence_to_index(l) for l in self.lines]
        
    def parse_words(self, lines):
        sls = lambda s: s.strip().lower().split(" ")
        words = ["<SOS>", "<EOS>"] + sorted(set(list(chain(*[sls(l) for l in lines]))))
        idx_word = dict(list(enumerate(words)))
        word_idx = dict(zip(words, list(range(len(words)))))
        
        return idx_word, word_idx
    
    def sentence_to_index(self, s):
        words = s.split(" ")
        indices = [self.word_idx[word] for word in words]
        return indices
    
    def index_to_sentence(self, indices):
        return " ".join(self.idx_word[idx] for idx in indices)

    
# you will use this class later, consider using ipython terminal
# to get used to initializing Parameters, Variables
class Linear(nn.Module):
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        
        # use rand(n) to get tensors to intitialize your weight matrix and bias tensor 
        # then use Parameter( ) to wrap them as Variables visible to module.parameters()
        
        
    def forward(self, input_var, use_relu=False):
        # standard linear layer, just an affine transform with no nonlinearity 
        #  for this lab
        # use torch.matmul not mm
        pass

In [ ]:
"""
encoder topology
-standard GRU, embed each word before submitting it.
-use GRUCell not GRU

Never use one hot encodings! Your input tensors to the GRU should have (1,1,hidden) size
-pytorch typically works with 3d (seq_len, batch_size, hidden_dim) tensors, unlike tensorflow's list of tensors.
"""

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, source_vocab_size, n_layers):
        super(Encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.vocab_size = source_vocab_size
        self.n_layers = n_layers
        
        #self.embed = nn.Embedding() # needs parameters
        #self.GRUCell = nn.GRUCell() # needs parameters
                
    def forward(self, input_variable):
        
        return hidden

"""
GRU is initialized to the number of layers
-run it one time step at a time using tensors of shape (1,1,hidden_size)
-use zero's as the initial hidden state

Use teacher forcing to initially establish word-word connections
-recommend around .5 to .7

without teacher forcing
next_input = tensor([[SOS]])
next_hidden = hidden
for i in 0..len(input_sequence):
  embed(next_input), h_i-1 -> GRU -> output, h_i
  output[-1] -> LinearLayer (to number of words in English corpus) -> SoftMax -> probabilities
  probabilities -> argmax -> next_input
  if next_input = EOS:
    break

with teacher forcing, helps to form one to one connections between words 
embedded = embed(reference_var)
next_hidden = hidden
for i in 0..len(embedded):
  embedded[i], h_i-1 -> GRU -> output, h_i
  output[-1] -> LinearLayer (to number of words in English corpus) -> SoftMax -> probabilities

Return the probabilities (for the loss) and predictions (for printing and easy later evaluation) whether or not using teacher forcing.
"""

class Decoder(nn.Module):
        def __init__(self, input_size, hidden_size, target_vocab_size, n_layers, max_target_length=30):
        super(Decoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.target_vocab_size = target_vocab_size
        self.n_layers = n_layers
        self.max_target_length = self.max_target_length
        
        # initialize GRU, a Linear Layer


    
    def forward(self, context, target_variable=None, encoder_outputs=None):
        use_teacher_forcing = target_variable or None
        
        # we may give you a module to implement the attention mechanism.        
        use_attention = encoder_outputs or None

In [ ]:
source_corpus = Corpus(source_lines)
target_corpus = Corpus(target_lines)
n_spanish = source_corpus.corpus_size
n_english = target_corpus.corpus_size

# accumulate gradients batch_size times before updating
epoch_length = np.minimum(len(training_pairs), 4000)
batch_size = 20 # rather iterations between optimizer updates
max_seq_len = 30 # watch the number of sentences drop
n_layers = 2
learning_rate = .01
print_every = batch_size
n_epochs = 30

input_size = 500
hidden_size = 500
teacher_forcing_ratio = .5
n_test = 5000


indexed_pairs = zip(source_corpus.training, target_corpus.training)

# support for training yet to come
encoder = Encoder(input_size, hidden_size, n_spanish, n_layers)
decoder = Decoder(input_size, hidden_size, n_spanish, n_layers, max_seq_len)

training_pairs = training_pairs[:-n_test]
testing_pairs = training_pairs[-n_test:]

def get_loss(output_probs, correct_indices):
    """ params:
         output_probs: a list of Variable (Not Tensor)
         correct_indices: a list or tensor of type int with the same length """
    
    # You can batch this part if you want by concatenating the output_probs
    # convert correct_indices to a (seq_len, 1) Tensor if batching or list of tensors
    # Use NLLoss as it takes probabilities
    # sanity check: loss should be a Variable

def print_output(teacher_forced, source_indices, predicted_indices, reference_indices):
    global source_corpus, target_corpus
    print("Iteration %d", end=" ")
    if teacher_forced:
        print("using teacher forcing")
    print ("In:    ", source_corpus.index_to_sentence(source_indices))
    print ("Out:   ", target_corpus.index_to_sentence(predicted_indices))
    print ("Ground:", target_corpus.index_to_sentence(predicted_indices))

# can combine encoder and decoder
def train(encoder, decoder, training_pairs, testing_pairs, 
                source_corpus, target_corpus, teacher_forcing_ratio, 
                epoch_size, learning_rate, batch_size, print_every):
    """
    You may want to lower the teacher forcing ratio as the number 
      of epochs progresses as it starts to learn word-word connections.
    
    In PyTorch some optimizers don't allow for decaying learning rates
    -thankfully initializing new optimizers is trivial
    -You may want to use a learning rate schedule instead of decay
    """

    # initialize the optimizers
    
    batched_loss = 0
    for i in range(n_epochs):
        for j in range(epoch_size):
            use_teacher_forcing = np.random.random() < teacher_forcing_ratio

            # consider whether or not to use teacher forcing on printing iterations
            # use_teacher_forcing = use_teacher_forcing or (i % print_every == 0)
            
            source_var, reference_var = np.random.choice(training_pairs)

            # convert source_var and reference_var to Tensors then Variables
            # run source_var through the encoder
            # if use_teacher_forcing input the target variable to decoder
            # compute the loss between the predicted probability distributions and the target indexed tensor

            # implement the batch update as shown in the spec

            # print results
            if i % print_every == 0:
                print_output(use_teacher_forcing, source_idc, predicted, target_idc)
            
    # run test iteration
    
    return encoder, decoder


"""
Help debugging / learning pytorch.
Consider using ipython terminal as part of your workflow and to get used to pytorch and the classes used in this lab.
pdb may be of interest to you.
"""